## Project 1: NYC Resteraunt Violations Cleaning Up
### Jacob Minkin

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
sns.set_style('darkgrid')

%matplotlib inline

## Set-Up

In [139]:
#Import Data
df = pd.read_csv('../data/Inspection.csv')

#Change Name of Column for easier searching
df.rename(columns = {'VIOLATION CODE':'CODE'}, inplace = True)

# Convert date into pandas date/time format
df['date'] = pd.to_datetime(df['INSPECTION DATE'])

# Create Dictionaries for Violation Codes and Resteraunt Names
code_list = df.CODE[df.CODE.isna() == False].unique()
violation_dicts = {}
for name in code_list:
    violation_dicts[name] = df.VIOLATION[df.CODE == name].unique()

Name_list = df.CAMIS[df.CAMIS.isna() == False].unique()
Name_dicts = {}
for name in Name_list:
    Name_dicts[name] = df.DBA[df.CAMIS == name].unique()

#Choose variables for final analysis
finalVariables = ['CAMIS','CUISINE','STREET', 'ZIPCODE', 'Community Board', 'BBL', 'BORO', 'date', 'CRITICAL FLAG']
fdf = df[finalVariables]

# df = df.drop(['Zip Codes', 'City Council Districts', 'Police Precincts', 'Location Point', 'Community Districts', 
#    'Borough Boundaries',  'GRADE DATE', 'PHONE', 'INSPECTION DATE', 'DBA', 'VIOLATION', 'RECORD DATE'], axis = 1)

# List of CONVERSION to categorical functions. 
to_convert = ['ZIPCODE', 'Community Board']
fdf[to_convert] = fdf[to_convert].astype('category')

c:\Users\jakem\anaconda3\lib\site-packages\pandas\core\frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [34]:
finalVariables = ['CAMIS','CUISINE','STREET', 'ZIPCODE', 'BBL', 'BORO', 'date', 'CRITICAL FLAG']
finalDataframe = df[finalVariables]

## Examine Missing Data

In [9]:
df.isnull().sum(axis = 0).sort_values(ascending = False)

GRADE               106150
SCORE                10915
CODE                  5681
BIN                   4561
CUISINE               4358
ACTION                4358
INSPECTION TYPE       4358
NTA                   3468
Community Board       3468
Council District      3468
Census Tract          3468
ZIPCODE               2933
BBL                    561
BUILDING               340
Longitude              298
Latitude               298
STREET                  26
BORO                     0
date                     0
CRITICAL FLAG            0
RECORD DATE              0
CAMIS                    0
dtype: int64

In [99]:
gk = df.sort_values(by=['date', 'GRADE'], ascending=False).groupby(by = ['CAMIS'], sort=False).nth(0)
gk.ACTION

CAMIS
50114345                  Establishment re-opened by DOHMH.
41603616                  Establishment re-opened by DOHMH.
50110842    Violations were cited in the following area(s).
50006741    Violations were cited in the following area(s).
50085148    Violations were cited in the following area(s).
                                 ...                       
50125351                                                NaN
50116200                                                NaN
50123266                                                NaN
50119602                                                NaN
50131767                                                NaN
Name: ACTION, Length: 28239, dtype: object

In [84]:
len(df.CAMIS.unique())

28239

In [54]:
df['RECORD DATE'].value_counts()

2/13/2023    209579
Name: RECORD DATE, dtype: int64

In [13]:
df.ACTION.value_counts()

Violations were cited in the following area(s).                                                                                       193989
Establishment Closed by DOHMH. Violations were cited in the following area(s) and those requiring immediate action were addressed.      7936
Establishment re-opened by DOHMH.                                                                                                       1985
No violations were recorded at the time of this inspection.                                                                             1299
Establishment re-closed by DOHMH.                                                                                                         12
Name: ACTION, dtype: int64

In [131]:
mice = {}
for k in Violations_dicts:
    if "04L" in Violations_dicts[k]:
         mice[k] = "04L"
    if "04K" in Violations_dicts[k]:
         mice[k] = "04K"

{50008108: '04L',
 41600677: '04K',
 41635743: '04K',
 40958104: '04L',
 50105967: '04L',
 41441583: '04L',
 40395087: '04K',
 50109092: '04L',
 50034739: '04L',
 50067634: '04L',
 50067182: '04L',
 50108990: '04L',
 50012215: '04L',
 50066104: '04L',
 41720851: '04L',
 40871961: '04L',
 50070745: '04L',
 50067979: '04L',
 50067257: '04L',
 50100549: '04L',
 50062556: '04L',
 41573408: '04L',
 50046390: '04L',
 50070862: '04L',
 50106052: '04L',
 50033770: '04L',
 41106321: '04L',
 50058321: '04L',
 41351011: '04L',
 41189848: '04L',
 40932307: '04L',
 50065696: '04K',
 40376515: '04L',
 41333314: '04K',
 50090918: '04L',
 50073372: '04L',
 50113510: '04K',
 50054846: '04L',
 50011205: '04L',
 50091173: '04L',
 41068607: '04L',
 40396532: '04L',
 50012113: '04L',
 40401000: '04L',
 41513756: '04L',
 40557901: '04L',
 40624607: '04L',
 50000910: '04L',
 41124419: '04L',
 50076792: '04L',
 50046232: '04L',
 40394401: '04L',
 41025692: '04L',
 50112611: '04L',
 41223224: '04L',
 41491686:

## Lets Focus on Cuisine

In [157]:
df.CUISINE.value_counts().sort_values(ascending=True).head(10)

Basque                 7
Armenian               8
Lebanese              13
New French            14
Czech                 14
Southwestern          21
Chilean               30
Iranian               34
Nuts/Confectionary    41
Californian           43
Name: CUISINE, dtype: int64

## Lets Focus on Critical Flags

In [162]:
df['CRITICAL FLAG']

0         Not Applicable
1         Not Applicable
2         Not Applicable
3         Not Applicable
4         Not Applicable
               ...      
209574          Critical
209575          Critical
209576      Not Critical
209577          Critical
209578      Not Critical
Name: CRITICAL FLAG, Length: 209579, dtype: object